In [1]:
import h2o
from h2o.automl import H2OAutoML
import random, os, sys
from datetime import datetime
import pandas as pd
import logging
import csv
import optparse
import time
import json
from distutils.util import strtobool
import psutil

In [18]:
data_path=None
all_variables=None
test_path=None
# target='search_term'
target='cnt'
nthreads=1 
min_mem_size=6 
run_time=1000
classification=False
scale=False
max_models=9    
model_path=None
balance_y=False 
balance_threshold=0.2
name=None 
server_path=None  
analysis=0

In [3]:
pct_memory=0.5
virtual_memory=psutil.virtual_memory()
min_mem_size=int(round(int(pct_memory*virtual_memory.available)/1073741824,0))
print(min_mem_size)

1


In [4]:
# Functions

def alphabet(n):
  alpha='0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'    
  str=''
  r=len(alpha)-1   
  while len(str)<n:
    i=random.randint(0,r)
    str+=alpha[i]   
  return str
  
  
def set_meta_data(analysis,run_id,server,data,test,model_path,target,run_time,classification,scale,model,balance,balance_threshold,name,path,nthreads,min_mem_size):
  m_data={}
  m_data['start_time'] = time.time()
  m_data['target']=target
  m_data['server_path']=server
  m_data['data_path']=data 
  m_data['test_path']=test
  m_data['max_models']=model
  m_data['run_time']=run_time
  m_data['run_id'] =run_id
  m_data['scale']=scale
  m_data['classification']=classification
  m_data['scale']=False
  m_data['model_path']=model_path
  m_data['balance']=balance
  m_data['balance_threshold']=balance_threshold
  m_data['project'] =name
  m_data['end_time'] = time.time()
  m_data['execution_time'] = 0.0
  m_data['run_path'] =path
  m_data['nthreads'] = nthreads
  m_data['min_mem_size'] = min_mem_size
  m_data['analysis'] = analysis
  return m_data


def dict_to_json(dct,n):
  j = json.dumps(dct, indent=4)
  f = open(n, 'w')
  print(j, file=f)
  f.close()
  
  
def stackedensemble(mod):
    coef_norm=None
    try:
      metalearner = h2o.get_model(mod.metalearner()['name'])
      coef_norm=metalearner.coef_norm()
    except:
      pass        
    return coef_norm

def stackedensemble_df(df):
    bm_algo={ 'GBM': None,'GLM': None,'DRF': None,'XRT': None,'Dee': None}
    for index, row in df.iterrows():
      if len(row['model_id'])>3:
        key=row['model_id'][0:3]
        if key in bm_algo:
          if bm_algo[key] is None:
                bm_algo[key]=row['model_id']
    bm=list(bm_algo.values()) 
    bm=list(filter(None.__ne__, bm))             
    return bm

def se_stats(modl):
    d={}
    d['algo']=modl.algo
    d['model_id']=modl.model_id   
    d['auc']=modl.auc()   
    d['roc']=modl.roc()
    d['mse']=modl.mse()   
    d['null_degrees_of_freedom']=modl.null_degrees_of_freedom()
    d['null_deviance']=modl.null_deviance()
    d['residual_degrees_of_freedom']=modl.residual_degrees_of_freedom()   
    d['residual_deviance']=modl.residual_deviance()
    d['rmse']=modl.rmse()
    return d

def get_model_by_algo(algo,models_dict):
    mod=None
    mod_id=None    
    for m in list(models_dict.keys()):
        if m[0:3]==algo:
            mod_id=m
            mod=h2o.get_model(m)      
    return mod,mod_id     
    
    
def gbm_stats(modl):
    d={}
    d['algo']=modl.algo
    d['model_id']=modl.model_id   
    d['varimp']=modl.varimp()  
    return d
    
    
def dl_stats(modl):
    d={}
    d['algo']=modl.algo
    d['model_id']=modl.model_id   
    d['varimp']=modl.varimp()  
    return d
    
    
def drf_stats(modl):
    d={}
    d['algo']=modl.algo
    d['model_id']=modl.model_id   
    d['varimp']=modl.varimp()  
    d['roc']=modl.roc()      
    return d
    
def xrt_stats(modl):
    d={}
    d['algo']=modl.algo
    d['model_id']=modl.model_id   
    d['varimp']=modl.varimp()  
    d['roc']=modl.roc()      
    return d
    
    
def glm_stats(modl):
    d={}
    d['algo']=modl.algo
    d['model_id']=modl.model_id   
    d['coef']=modl.coef()  
    d['coef_norm']=modl.coef_norm()      
    return d
    
def model_performance_stats(perf):
    d={}
    try:    
      d['mse']=perf.mse()
    except:
      pass      
    try:    
      d['rmse']=perf.rmse() 
    except:
      pass      
    try:    
      d['null_degrees_of_freedom']=perf.null_degrees_of_freedom()
    except:
      pass      
    try:    
      d['residual_degrees_of_freedom']=perf.residual_degrees_of_freedom()
    except:
      pass      
    try:    
      d['residual_deviance']=perf.residual_deviance() 
    except:
      pass      
    try:    
      d['null_deviance']=perf.null_deviance() 
    except:
      pass      
    try:    
      d['aic']=perf.aic() 
    except:
      pass      
    try:
      d['logloss']=perf.logloss() 
    except:
      pass    
    try:
      d['auc']=perf.auc()
    except:
      pass  
    try:
      d['gini']=perf.gini()
    except:
      pass    
    return d
    
def impute_missing_values(df, x, scal=False):
    # determine column types
    ints, reals, enums = [], [], []
    for key, val in df.types.items():
        if key in x:
            if val == 'enum':
                enums.append(key)
            elif val == 'int':
                ints.append(key)            
            else: 
                reals.append(key)    
    _ = df[reals].impute(method='mean')
    _ = df[ints].impute(method='median')
    if scal:
        df[reals] = df[reals].scale()
        df[ints] = df[ints].scale()    
    return


def get_independent_variables(df, targ):
    C = [name for name in df.columns if name != targ]
    # determine column types
    ints, reals, enums = [], [], []
    for key, val in df.types.items():
        if key in C:
            if val == 'enum':
                enums.append(key)
            elif val == 'int':
                ints.append(key)            
            else: 
                reals.append(key)    
    x=ints+enums+reals
    return x
    
def get_all_variables_csv(i):
    ivd={}
    try:
      iv = pd.read_csv(i,header=None)
    except:
      sys.exit(1)    
    col=iv.values.tolist()[0]
    dt=iv.values.tolist()[1]
    i=0
    for c in col:
      ivd[c.strip()]=dt[i].strip()
      i+=1        
    return ivd
    
    

def check_all_variables(df,dct,y=None):     
    targ=list(dct.keys())     
    for key, val in df.types.items():
        if key in targ:
          if dct[key] not in ['real','int','enum']:                      
            targ.remove(key)  
    for key, val in df.types.items():
        if key in targ:            
          if dct[key] != val:
            print('convert ',key,' ',dct[key],' ',val)
            if dct[key]=='enum':
                try:
                  df[key] = df[key].asfactor() 
                except:
                  targ.remove(key)                 
            if dct[key]=='int': 
                try:                
                  df[key] = df[key].asnumeric() 
                except:
                  targ.remove(key)                  
            if dct[key]=='real':
                try:                
                  df[key] = df[key].asnumeric()  
                except:
                  targ.remove(key)                  
    if y is None:
      y=df.columns[-1] 
    if y in targ:
      targ.remove(y)
    else:
      y=targ.pop()            
    return targ    
    
def predictions(mod,data,run_id):
    test = h2o.import_file(data)
    mod_perf=mod_best.model_performance(test)
              
    stats_test={}
    stats_test=model_performance_stats(mod_perf)

    n=run_id+'_test_stats.json'
    dict_to_json(stats_test,n) 

    try:    
      cf=mod_perf.confusion_matrix(metrics=["f1","f2","f0point5","accuracy","precision","recall","specificity","absolute_mcc","min_per_class_accuracy","mean_per_class_accuracy"])
      cf_df=cf[0].table.as_data_frame()
      cf_df.to_csv(run_id+'_test_confusion_matrix.csv')
    except:
      pass

    predictions = mod_best.predict(test)
    predictions_df=test.cbind(predictions).as_data_frame() 
    predictions_df.to_csv(run_id+'_predictions.csv')
    return

def predictions_test(mod,test,run_id):
    mod_perf=mod_best.model_performance(test)          
    stats_test={}
    stats_test=model_performance_stats(mod_perf)
    n=run_id+'_test_stats.json'
    dict_to_json(stats_test,n) 
    try:
      cf=mod_perf.confusion_matrix()
#      cf=mod_perf.confusion_matrix(metrics=["f1","f2","f0point5","accuracy","precision","recall","specificity","absolute_mcc","min_per_class_accuracy","mean_per_class_accuracy"])
      cf_df=cf.table.as_data_frame()
      cf_df.to_csv(run_id+'_test_confusion_matrix.csv')
    except:
      pass
    predictions = mod_best.predict(test)    
    predictions_df=test.cbind(predictions).as_data_frame() 
    predictions_df.to_csv(run_id+'_predictions.csv')
    return predictions

def check_X(x,df):
    for name in x:
        if name not in df.columns:
          x.remove(name)  
    return x    
    
    
def get_stacked_ensemble(lst):
    se=None
    for model in model_set:
      if 'BestOfFamily' in model:
        se=model
    if se is None:     
      for model in model_set:
        if 'AllModels'in model:
          se=model           
    return se       
    
def get_variables_types(df):
    d={}
    for key, val in df.types.items():
        d[key]=val           
    return d    
    
#  End Functions


In [5]:
all_variables=None

In [6]:
data_path='/Users/newzysharma/Desktop/Desktop/info6105/INFO6105-FinalProject/INFO6105-FinalProject/MetaData/hour.csv'

In [7]:
run_id=alphabet(5)
if server_path==None:
  server_path=os.path.abspath(os.curdir)
os.chdir(server_path) 
run_dir = os.path.join(server_path,run_id)
os.mkdir(run_dir)
os.chdir(run_dir)    

# run_id to std out
print (run_id)

am1d1


In [8]:
logfile=run_id+'_autoh2o_log.zip'
logs_path=os.path.join(run_dir,'logs')
print(logs_path,' ',logfile)

/Users/newzysharma/Desktop/Desktop/info6105/INFO6105-FinalProject/INFO6105-FinalProject/MetaData/am1d1/logs   am1d1_autoh2o_log.zip


In [9]:
# 65535 Highest port no
port_no=random.randint(5555,55555)
h2o.init(strict_version_check=False,min_mem_size_GB=min_mem_size,port=port_no)

Checking whether there is an H2O instance running at http://localhost:42630 ..... not found.
Attempting to start a local H2O server...
  Java Version: java version "11.0.2" 2019-01-15 LTS; Java(TM) SE Runtime Environment 18.9 (build 11.0.2+9-LTS); Java HotSpot(TM) 64-Bit Server VM 18.9 (build 11.0.2+9-LTS, mixed mode)
  Starting server from /anaconda3/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/8c/swkczny90t36p2798_2khxfh0000gn/T/tmpka1tkgud
  JVM stdout: /var/folders/8c/swkczny90t36p2798_2khxfh0000gn/T/tmpka1tkgud/h2o_newzysharma_started_from_python.out
  JVM stderr: /var/folders/8c/swkczny90t36p2798_2khxfh0000gn/T/tmpka1tkgud/h2o_newzysharma_started_from_python.err
  Server is running at http://127.0.0.1:42630
Connecting to H2O server at http://127.0.0.1:42630 ... successful.


H2O cluster uptime:,01 secs
H2O cluster timezone:,America/New_York
H2O data parsing timezone:,UTC
H2O cluster version:,3.24.0.1
H2O cluster version age:,17 days
H2O cluster name:,H2O_from_python_newzysharma_1pbdzd
H2O cluster total nodes:,1
H2O cluster free memory:,2 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"accepting new members, healthy"


In [10]:
# meta data
meta_data = set_meta_data(analysis, run_id,server_path,data_path,test_path,model_path,target,run_time,classification,scale,max_models,balance_y,balance_threshold,name,run_dir,nthreads,min_mem_size)
print(meta_data)


{'start_time': 1555638484.295697, 'target': None, 'server_path': '/Users/newzysharma/Desktop/Desktop/info6105/INFO6105-FinalProject/INFO6105-FinalProject/MetaData', 'data_path': '/Users/newzysharma/Desktop/Desktop/info6105/INFO6105-FinalProject/INFO6105-FinalProject/MetaData/hour.csv', 'test_path': None, 'max_models': 9, 'run_time': 1000, 'run_id': 'am1d1', 'scale': False, 'classification': False, 'model_path': None, 'balance': False, 'balance_threshold': 0.2, 'project': None, 'end_time': 1555638484.295706, 'execution_time': 0.0, 'run_path': '/Users/newzysharma/Desktop/Desktop/info6105/INFO6105-FinalProject/INFO6105-FinalProject/MetaData/am1d1', 'nthreads': 1, 'min_mem_size': 1, 'analysis': 0}


In [11]:
print(data_path)

/Users/newzysharma/Desktop/Desktop/info6105/INFO6105-FinalProject/INFO6105-FinalProject/MetaData/hour.csv


In [12]:
df = h2o.import_file(data_path)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [13]:
df.head()

instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
1,2011-01-01 00:00:00,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0,3,13,16
2,2011-01-01 00:00:00,1,0,1,1,0,6,0,1,0.22,0.2727,0.8,0,8,32,40
3,2011-01-01 00:00:00,1,0,1,2,0,6,0,1,0.22,0.2727,0.8,0,5,27,32
4,2011-01-01 00:00:00,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0,3,10,13
5,2011-01-01 00:00:00,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0,0,1,1
6,2011-01-01 00:00:00,1,0,1,5,0,6,0,2,0.24,0.2576,0.75,0.0896,0,1,1
7,2011-01-01 00:00:00,1,0,1,6,0,6,0,1,0.22,0.2727,0.8,0,2,0,2
8,2011-01-01 00:00:00,1,0,1,7,0,6,0,1,0.2,0.2576,0.86,0,1,2,3
9,2011-01-01 00:00:00,1,0,1,8,0,6,0,1,0.24,0.2879,0.75,0,1,7,8
10,2011-01-01 00:00:00,1,0,1,9,0,6,0,1,0.32,0.3485,0.76,0,8,6,14


In [14]:
df.describe()

Rows:17379
Cols:17




,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
type,int,time,int,int,int,int,int,int,int,int,real,real,real,real,int,int,int
mins,1.0,1293840000000.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.02,0.0,0.0,0.0,0.0,0.0,1.0
mean,8690.0,1325477314552.0461,2.501639910236492,0.5025605615973301,6.5377754761493785,11.546751826917548,0.028770355026181024,3.003682605443351,0.6827205247712756,1.425283387997008,0.4969871684216584,0.47577510213476026,0.6272288394038784,0.1900976063064618,35.67621842453536,153.78686920996606,189.4630876345014
maxs,17379.0,1356912000000.0,4.0,1.0,12.0,23.0,1.0,6.0,1.0,4.0,1.0,1.0,1.0,0.8507,367.0,886.0,977.0
sigma,5017.029499614288,18150225217.779854,1.1069181394480765,0.5000078290910197,3.438775713750168,6.914405095264493,0.16716527638437123,2.005771456110988,0.4654306335238829,0.6393568777542534,0.19255612124972193,0.17185021563535932,0.19292983406291514,0.1223402285727905,49.30503038705309,151.35728591258314,181.38759909186476
zeros,0,0,0,8645,0,726,16879,2502,5514,0,0,2,22,2180,1581,24,0
missing,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,1.0,2011-01-01 00:00:00,1.0,0.0,1.0,0.0,0.0,6.0,0.0,1.0,0.24,0.2879,0.81,0.0,3.0,13.0,16.0
1,2.0,2011-01-01 00:00:00,1.0,0.0,1.0,1.0,0.0,6.0,0.0,1.0,0.22,0.2727,0.8,0.0,8.0,32.0,40.0
2,3.0,2011-01-01 00:00:00,1.0,0.0,1.0,2.0,0.0,6.0,0.0,1.0,0.22,0.2727,0.8,0.0,5.0,27.0,32.0


In [15]:
'''
import sys
sys.stdout = open('describe.txt', 'w')
print ('test')
'''

"\nimport sys\nsys.stdout = open('describe.txt', 'w')\nprint ('test')\n"

In [19]:
# dependent variable
# assign target and inputs for classification or regression
if target==None:
  target=df.columns[5]   
y = target

In [20]:
print(y)

cnt


In [21]:
print(all_variables)

None


In [22]:
if all_variables is not None:
  ivd=get_all_variables_csv(all_variables)
  print(ivd)    
  X=check_all_variables(df,ivd,y)
  print(X)

In [23]:
# independent variables

X = []  
if all_variables is None:
  X=get_independent_variables(df, target)  
else: 
  ivd=get_all_variables_csv(all_variables)    
  X=check_all_variables(df, ivd)


X=check_X(X,df)


# Add independent variables

meta_data['X']=X  


# impute missing values

_=impute_missing_values(df,X, scale)

In [24]:
if analysis == 3:
  classification=False
elif analysis == 2:
  classification=True
elif analysis == 1:
  classification=True

In [25]:
# Force target to be factors
# Only 'int' or 'string' are allowed for asfactor(), got Target (Total orders):real 

if classification:
    df[y] = df[y].asfactor()

In [26]:
def check_y(y,df):
  ok=False
  C = [name for name in df.columns if name == y]
  for key, val in df.types.items():
    if key in C:
      if val in ['real','int','enum']:        
        ok=True         
  return ok

In [27]:
ok=check_y(y,df)
if not ok:
    print(ok)

In [28]:
print(ok)

True


In [29]:
classification=True
if classification:
    print(df[y].levels())

[]


In [30]:
print(df[y].levels())

[]


In [31]:
allV=get_variables_types(df)
allV

{'instant': 'int',
 'dteday': 'time',
 'season': 'int',
 'yr': 'int',
 'mnth': 'int',
 'hr': 'int',
 'holiday': 'int',
 'weekday': 'int',
 'workingday': 'int',
 'weathersit': 'int',
 'temp': 'real',
 'atemp': 'real',
 'hum': 'real',
 'windspeed': 'real',
 'casual': 'int',
 'registered': 'int',
 'cnt': 'int'}

In [32]:
meta_data['variables']=allV

In [33]:
# Set up AutoML

aml = H2OAutoML(max_runtime_secs=run_time,project_name = name)

In [34]:
model_start_time = time.time()

In [35]:
aml.train(x=X,y=y,training_frame=df)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [36]:
meta_data['model_execution_time'] = time.time() - model_start_time

In [37]:
execution_time =  time.time() - model_start_time
meta_data['model_execution_time_sec'] = execution_time
print("Execution time for ", run_time,"sec =  ",meta_data['model_execution_time_sec'])

Execution time for  1000 sec =   1140.7439978122711


In [38]:
# get leaderboard
aml_leaderboard_df=aml.leaderboard.as_data_frame()

In [39]:
aml_leaderboard_df

,model_id,mean_residual_deviance,rmse,mse,mae,rmsle
0,XGBoost_1_AutoML_20190418_214850,19.214101,4.383389,19.214101,2.634751,0.054888
1,GBM_1_AutoML_20190418_214850,25.332040,5.033095,25.332040,3.227131,NaN
2,XGBoost_2_AutoML_20190418_214850,27.239170,5.219116,27.239170,3.262814,NaN
3,XGBoost_3_AutoML_20190418_214850,27.944545,5.286260,27.944545,3.475809,0.099999
4,GBM_3_AutoML_20190418_214850,31.736674,5.633531,31.736674,3.454525,0.073510
5,GLM_grid_1_AutoML_20190418_214850_model_1,38.387049,6.195728,38.387049,4.451969,NaN
6,GBM_2_AutoML_20190418_214850,38.628741,6.215202,38.628741,4.064625,NaN
7,StackedEnsemble_AllModels_AutoML_20190418_214850,41.682334,6.456186,41.682334,4.399957,0.234378
8,DRF_1_AutoML_20190418_214850,46.675148,6.831921,46.675148,3.714254,0.080411
9,StackedEnsemble_BestOfFamily_AutoML_20190418_2...,110.001774,10.488173,110.001774,7.815701,0.380275


In [40]:
# STart best model as first model

model_set=aml_leaderboard_df['model_id']
mod_best=h2o.get_model(model_set[0])

In [41]:
mod_best._id

'XGBoost_1_AutoML_20190418_214850'

In [42]:
# Get stacked ensemble  
se=get_stacked_ensemble(model_set)

In [43]:
print(se)

StackedEnsemble_BestOfFamily_AutoML_20190418_214850


In [44]:
dir(mod_best)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_bc',
 '_bcin',
 '_check_and_save_parm',
 '_check_targets',
 '_compute_algo',
 '_end_time',
 '_estimator_type',
 '_future',
 '_get_metrics',
 '_have_mojo',
 '_have_pojo',
 '_id',
 '_is_xvalidated',
 '_job',
 '_keyify_if_h2oframe',
 '_make_model',
 '_metrics_class',
 '_model_json',
 '_parms',
 '_plot',
 '_requires_training_frame',
 '_resolve_model',
 '_run_time',
 '_start_time',
 '_train',
 '_verify_training_frame_params',
 '_xval_keys',
 'actual_params',
 'aic',
 'algo',
 'auc',
 'available',
 'backend',
 'biases',
 'booster',
 'categorical_encoding',
 'catoffsets',
 'coef',
 'coef_norm',
 'col_sample_rat

In [45]:
mod_best._id

'XGBoost_1_AutoML_20190418_214850'

In [46]:
mod_best._get_metrics

<function h2o.model.model_base.ModelBase._get_metrics(o, train, valid, xval)>

In [47]:
type(mod_best)

h2o.estimators.xgboost.H2OXGBoostEstimator

In [48]:
mods=mod_best.coef_norm
print(mods)

Model Details
H2OXGBoostEstimator :  XGBoost
Model Key:  XGBoost_1_AutoML_20190418_214850


ModelMetricsRegression: xgboost
** Reported on train data. **

MSE: 1.0863112926632854
RMSE: 1.0422625833556942
MAE: 0.7524023069080258
RMSLE: 0.026106630621527304
Mean Residual Deviance: 1.0863112926632854

ModelMetricsRegression: xgboost
** Reported on cross-validation data. **

MSE: 19.21410106621536
RMSE: 4.383389221391977
MAE: 2.6347511218427426
RMSLE: 0.054887779739166816
Mean Residual Deviance: 19.21410106621536
Cross-Validation Metrics Summary: 


,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
mae,2.6347408,0.0680855,2.6200283,2.6675117,2.7307901,2.6987867,2.4565876
mean_residual_deviance,19.21389,1.6949317,17.639582,19.431736,21.773092,21.68898,15.536058
mse,19.21389,1.6949317,17.639582,19.431736,21.773092,21.68898,15.536058
r2,0.9994161,0.0000514,0.9994645,0.9994139,0.9993344,0.999342,0.9995252
residual_deviance,19.21389,1.6949317,17.639582,19.431736,21.773092,21.68898,15.536058
rmse,4.3745966,0.1959525,4.19995,4.408144,4.666165,4.6571426,3.9415808
rmsle,0.0547402,0.0028432,0.0570302,0.0558823,0.0478816,0.0597229,0.0531840


Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance
,2019-04-18 21:56:31,7 min 41.505 sec,0.0,261.9286476,188.9630876,68606.6164192
,2019-04-18 21:56:31,7 min 41.631 sec,5.0,204.9477789,146.5903879,42003.5920672
,2019-04-18 21:56:31,7 min 41.744 sec,10.0,159.3670700,113.6586630,25397.8630023
,2019-04-18 21:56:32,7 min 41.856 sec,15.0,123.9921689,88.1119079,15374.0579585
,2019-04-18 21:56:32,7 min 41.994 sec,20.0,97.0797121,68.4530253,9424.4705084
---,---,---,---,---,---,---
,2019-04-18 21:57:45,8 min 55.188 sec,485.0,1.0715175,0.7717919,1.1481498
,2019-04-18 21:57:46,8 min 56.578 sec,490.0,1.0617651,0.7656627,1.1273452
,2019-04-18 21:57:48,8 min 58.112 sec,495.0,1.0521693,0.7589569,1.1070603
,2019-04-18 21:57:49,8 min 59.607 sec,500.0,1.0436282,0.7533147,1.0891598



See the whole table with table.as_data_frame()
Variable Importances: 


variable,relative_importance,scaled_importance,percentage
registered,2660129024.0000000,1.0,0.7615605
casual,597958656.0000000,0.2247856,0.1711878
hr,130871832.0000000,0.0491976,0.0374669
instant,30420522.0000000,0.0114357,0.0087090
workingday,24261744.0000000,0.0091205,0.0069458
weekday,19920988.0000000,0.0074887,0.0057031
dteday,10809360.0000000,0.0040635,0.0030946
temp,7532853.0,0.0028318,0.0021566
atemp,6647484.5,0.0024989,0.0019031
hum,995252.9375000,0.0003741,0.0002849


<bound method ModelBase.coef_norm of >


In [49]:
'''
mod_best.aic()
 mod_best.algo
 mod_best.coef()
 mod_best.coef_norm()
 mod_best.confusion_matrix()
 mod_best.cross_validation_fold_assignment()
 mod_best.cross_validation_holdout_predictions()
 mod_best.cross_validation_metrics_summary()
 mod_best.cross_validation_models()
 mod_best.cross_validation_predictions()
 mod_best.deepfeatures()
 mod_best.default_params()
 mod_best.download_mojo()
 mod_best.download_pojo()
 mod_best.fit()
 mod_best.full_parameters()
 mod_best.get_params()
 mod_best.get_xval_models()
 mod_best.gini()
 mod_best.have_mojo()
 mod_best.have_pojo()
 mod_best.hit_ratio_table()
 mod_best.is_cross_validated()
 mod_best.join()
 mod_best.keep_levelone_frame()
 mod_best.levelone_frame_id()
 mod_best.logloss()
 mod_best.mae()
 mod_best.mean_per_class_error()
 mod_best.mean_residual_deviance()
 mod_best.metalearner()
 mod_best.metalearner_algorithm()
 mod_best.metalearner_fold_assignment()
 mod_best.metalearner_fold_column()
 mod_best.metalearner_nfolds()
 mod_best.metalearner_params()
 mod_best.mixin()
 mod_best.model_id()
 mod_best.model_performance()
 mod_best.mse()
 mod_best.normmul()
 mod_best.normsub()
 mod_best.null_degrees_of_freedom()
 mod_best.null_deviance()
 mod_best.params()
 mod_best.parms()
 mod_best.partial_plot()
 mod_best.plot()
 mod_best.pprint_coef()
 mod_best.predict()
 mod_best.predict_leaf_node_assignment()
 mod_best.r2()
 mod_best.residual_degrees_of_freedom()
 mod_best.residual_deviance()
 mod_best.respmul()
 mod_best.response_column()
 mod_best.respsub()
 mod_best.rmse()
 mod_best.rmsle()
 mod_best.rotation()
 mod_best.save_model_details()
 mod_best.save_mojo()
 mod_best.score_history()
 mod_best.scoring_history()
 mod_best.seed()
 mod_best.set_params()
 mod_best.show()
 mod_best.start()
 mod_best.std_coef_plot()
 mod_best.summary()
 mod_best.train()
 mod_best.training_frame()
 mod_best.type()
 mod_best.validation_frame()
 mod_best.varimp()
 mod_best.varimp_plot()
 mod_best.weights()
 mod_best.xval_keys()
 mod_best.xvalsmod_best()
 '''


'\nmod_best.aic()\n mod_best.algo\n mod_best.coef()\n mod_best.coef_norm()\n mod_best.confusion_matrix()\n mod_best.cross_validation_fold_assignment()\n mod_best.cross_validation_holdout_predictions()\n mod_best.cross_validation_metrics_summary()\n mod_best.cross_validation_models()\n mod_best.cross_validation_predictions()\n mod_best.deepfeatures()\n mod_best.default_params()\n mod_best.download_mojo()\n mod_best.download_pojo()\n mod_best.fit()\n mod_best.full_parameters()\n mod_best.get_params()\n mod_best.get_xval_models()\n mod_best.gini()\n mod_best.have_mojo()\n mod_best.have_pojo()\n mod_best.hit_ratio_table()\n mod_best.is_cross_validated()\n mod_best.join()\n mod_best.keep_levelone_frame()\n mod_best.levelone_frame_id()\n mod_best.logloss()\n mod_best.mae()\n mod_best.mean_per_class_error()\n mod_best.mean_residual_deviance()\n mod_best.metalearner()\n mod_best.metalearner_algorithm()\n mod_best.metalearner_fold_assignment()\n mod_best.metalearner_fold_column()\n mod_best.met

In [50]:
    bm=stackedensemble_df(aml_leaderboard_df)

In [51]:
bm

['GBM_1_AutoML_20190418_214850',
 'GLM_grid_1_AutoML_20190418_214850_model_1',
 'DRF_1_AutoML_20190418_214850']

In [52]:
aml_leaderboard_df

,model_id,mean_residual_deviance,rmse,mse,mae,rmsle
0,XGBoost_1_AutoML_20190418_214850,19.214101,4.383389,19.214101,2.634751,0.054888
1,GBM_1_AutoML_20190418_214850,25.332040,5.033095,25.332040,3.227131,NaN
2,XGBoost_2_AutoML_20190418_214850,27.239170,5.219116,27.239170,3.262814,NaN
3,XGBoost_3_AutoML_20190418_214850,27.944545,5.286260,27.944545,3.475809,0.099999
4,GBM_3_AutoML_20190418_214850,31.736674,5.633531,31.736674,3.454525,0.073510
5,GLM_grid_1_AutoML_20190418_214850_model_1,38.387049,6.195728,38.387049,4.451969,NaN
6,GBM_2_AutoML_20190418_214850,38.628741,6.215202,38.628741,4.064625,NaN
7,StackedEnsemble_AllModels_AutoML_20190418_214850,41.682334,6.456186,41.682334,4.399957,0.234378
8,DRF_1_AutoML_20190418_214850,46.675148,6.831921,46.675148,3.714254,0.080411
9,StackedEnsemble_BestOfFamily_AutoML_20190418_2...,110.001774,10.488173,110.001774,7.815701,0.380275


In [53]:
#  Get best_models and coef_norm()
best_models={}
best_models=stackedensemble(mod_best)
bm=[]
if best_models is not None: 
  if 'Intercept' in best_models.keys():
    del best_models['Intercept']
  bm=list(best_models.keys())
else:
  best_models={}
  bm=stackedensemble_df(aml_leaderboard_df)   
  for b in bm:   
    best_models[b]=None

if mod_best.model_id not in bm:
    bm.append(mod_best.model_id)

In [54]:
bm

['GBM_1_AutoML_20190418_214850',
 'GLM_grid_1_AutoML_20190418_214850_model_1',
 'DRF_1_AutoML_20190418_214850',
 'XGBoost_1_AutoML_20190418_214850']

In [55]:
# Best of Family leaderboard

aml_leaderboard_df=aml_leaderboard_df.loc[aml_leaderboard_df['model_id'].isin(bm)]

In [56]:
aml_leaderboard_df

,model_id,mean_residual_deviance,rmse,mse,mae,rmsle
0,XGBoost_1_AutoML_20190418_214850,19.214101,4.383389,19.214101,2.634751,0.054888
1,GBM_1_AutoML_20190418_214850,25.332040,5.033095,25.332040,3.227131,NaN
5,GLM_grid_1_AutoML_20190418_214850_model_1,38.387049,6.195728,38.387049,4.451969,NaN
8,DRF_1_AutoML_20190418_214850,46.675148,6.831921,46.675148,3.714254,0.080411


In [57]:
# save leaderboard
leaderboard_stats=run_id+'_leaderboard.csv'
aml_leaderboard_df.to_csv(leaderboard_stats)

In [58]:
top=aml_leaderboard_df.iloc[0]['model_id']
print(top)

XGBoost_1_AutoML_20190418_214850


In [59]:
mod_best=h2o.get_model(top)
print(mod_best._id)
print(mod_best.algo)

XGBoost_1_AutoML_20190418_214850
xgboost


In [60]:
meta_data['mod_best']=mod_best._id
meta_data['mod_best_algo']=mod_best.algo

In [61]:
meta_data['models']=bm

In [62]:
models_path=os.path.join(run_dir,'models')
for mod in bm:
  try:   
    m=h2o.get_model(mod) 
    h2o.save_model(m, path = models_path)
  except:    
    pass

In [63]:
print(models_path)

/Users/newzysharma/Desktop/Desktop/info6105/INFO6105-FinalProject/INFO6105-FinalProject/MetaData/am1d1/models


In [64]:
# GBM
 
mod,mod_id=get_model_by_algo("GBM",best_models)
if mod is not None:
    try:     
        sh_df=mod.scoring_history()
        sh_df.to_csv(run_id+'_gbm_scoring_history.csv') 
    except:
        pass   
    try:     
        stats_gbm={}
        stats_gbm=gbm_stats(mod)
        n=run_id+'_gbm_stats.json'
        dict_to_json(stats_gbm,n)
        print(stats_gbm)
    except:
        pass

{'algo': 'gbm', 'model_id': 'GBM_1_AutoML_20190418_214850', 'varimp': [('registered', 1191396480.0, 1.0, 0.4421670515252589), ('casual', 981601856.0, 0.823908642066829, 0.3643052549888697), ('hr', 307487392.0, 0.2580898946419583, 0.11411884774229943), ('workingday', 86338992.0, 0.07246873181965419, 0.03204328547647121), ('dteday', 36148732.0, 0.03034147960551302, 0.013416002576083467), ('instant', 30986518.0, 0.026008569372305012, 0.01150013243374226), ('atemp', 26250434.0, 0.0220333318426457, 0.00974241337614025), ('temp', 24253822.0, 0.02035747327371657, 0.009001403933943516), ('yr', 2992835.75, 0.002512040114471381, 0.0011107413707289843), ('season', 1974824.625, 0.0016575713107696943, 0.0007329234191758944), ('hum', 1771373.375, 0.0014868042710685196, 0.0006574158607335291), ('mnth', 1641412.75, 0.0013777216716302535, 0.0006091831293672002), ('weekday', 862785.1875, 0.000724179735280064, 0.0003202084183231284), ('weathersit', 477448.0, 0.00040074652562344316, 0.00017719690964390947

In [65]:
if mod is not None:
    try:    
        sh_df=mod.scoring_history()
        sh_df.to_csv(run_id+'_dl_scoring_history.csv') 
    except:
        pass 
    try:
        stats_dl={}
        stats_dl=dl_stats(mod)
        n=run_id+'_dl_stats.json'
        dict_to_json(stats_dl,n)
        print(stats_dl)
    except:
        pass    
    try:
        cf=mod.confusion_matrix()    
        cf_df.to_csv(run_id+'_dl_confusion_matrix.csv')
    except:
        pass

{'algo': 'gbm', 'model_id': 'GBM_1_AutoML_20190418_214850', 'varimp': [('registered', 1191396480.0, 1.0, 0.4421670515252589), ('casual', 981601856.0, 0.823908642066829, 0.3643052549888697), ('hr', 307487392.0, 0.2580898946419583, 0.11411884774229943), ('workingday', 86338992.0, 0.07246873181965419, 0.03204328547647121), ('dteday', 36148732.0, 0.03034147960551302, 0.013416002576083467), ('instant', 30986518.0, 0.026008569372305012, 0.01150013243374226), ('atemp', 26250434.0, 0.0220333318426457, 0.00974241337614025), ('temp', 24253822.0, 0.02035747327371657, 0.009001403933943516), ('yr', 2992835.75, 0.002512040114471381, 0.0011107413707289843), ('season', 1974824.625, 0.0016575713107696943, 0.0007329234191758944), ('hum', 1771373.375, 0.0014868042710685196, 0.0006574158607335291), ('mnth', 1641412.75, 0.0013777216716302535, 0.0006091831293672002), ('weekday', 862785.1875, 0.000724179735280064, 0.0003202084183231284), ('weathersit', 477448.0, 0.00040074652562344316, 0.00017719690964390947

In [66]:
# DRF

mod,mod_id=get_model_by_algo("DRF",best_models)
if mod is not None:
    try:     
         sh_df=mod.scoring_history()
         sh_df.to_csv(run_id+'_drf_scoring_history.csv') 
    except:
         pass  
    try: 
         stats_drf={}
         stats_drf=drf_stats(mod)
         n=run_id+'_drf_stats.json'
         dict_to_json(stats_drf,n)
         print(stats_drf)
    except:
         pass

In [67]:
# XRT

mod,mod_id=get_model_by_algo("XRT",best_models)
if mod is not None:
    try:     
         sh_df=mod.scoring_history()
         sh_df.to_csv(run_id+'_xrt_scoring_history.csv')
    except:
         pass     
    try:        
         stats_xrt={}
         stats_xrt=xrt_stats(mod)
         n=run_id+'_xrt_stats.json'
         dict_to_json(stats_xrt,n)
         print(stats_xrt)
    except:
         pass

In [68]:
# GLM

mod,mod_id=get_model_by_algo("GLM",best_models)
if mod is not None:
    try:     
         stats_glm={}
         stats_glm=glm_stats(mod)
         n=run_id+'_glm_stats.json'
         dict_to_json(stats_glm,n)
         print(stats_glm)
    except:
         pass

{'algo': 'glm', 'model_id': 'GLM_grid_1_AutoML_20190418_214850_model_1', 'coef': {'Intercept': -28.9024709383948, 'instant': 7.893290280699375e-05, 'dteday': 2.1090705458180994e-11, 'season': 0.6622390083830808, 'yr': 1.4720625025624565, 'mnth': -0.08848850614264787, 'hr': 0.25445121769334494, 'holiday': -0.42204010084117155, 'weekday': 0.03745819439023113, 'workingday': 1.6880335477332526, 'weathersit': -0.239489670411643, 'temp': 2.7092038106611542, 'atemp': 4.69252713516861, 'hum': -4.916196250005459, 'windspeed': 1.4855252234572285, 'casual': 1.003627490697193, 'registered': 0.9589464264146543}, 'coef_norm': {'Intercept': 189.46308763450153, 'instant': 0.3960087018728751, 'dteday': 0.3828010540678439, 'season': 0.733044371029339, 'yr': 0.7360427761925474, 'mnth': -0.3042921258693701, 'hr': 1.7593787961151188, 'holiday': -0.07055045010240235, 'weekday': 0.07513257710538233, 'workingday': 0.7856625235310555, 'weathersit': -0.15311936792878328, 'temp': 0.521673777455878, 'atemp': 0.80

In [69]:
# split into training and test for showing how to predict
train, test = df.split_frame([0.8])

In [70]:
predictions_df=predictions_test(mod_best,test,run_id)

xgboost prediction progress: |████████████████████████████████████████████| 100%


In [71]:
predictions_df.head()

predict
16.0361
12.3063
17.0025
9.14764
8.1211
53.502
69.7953
58.7203
22.7269
31.1814


In [72]:
predictions_df.describe()

Rows:3437
Cols:1




,predict
type,real
mins,0.6442035436630249
mean,189.31918917282883
maxs,970.2227172851562
sigma,182.7975498260944
zeros,0
missing,0
0,16.036054611206055
1,12.306280136108398
2,17.002532958984375


In [73]:
# Update and save meta data

meta_data['end_time'] = time.time()
meta_data['execution_time'] = meta_data['end_time'] - meta_data['start_time']
  
n=run_id+'_meta_data.json'
dict_to_json(meta_data,n)

In [74]:
meta_data

{'start_time': 1555638484.295697,
 'target': None,
 'server_path': '/Users/newzysharma/Desktop/Desktop/info6105/INFO6105-FinalProject/INFO6105-FinalProject/MetaData',
 'data_path': '/Users/newzysharma/Desktop/Desktop/info6105/INFO6105-FinalProject/INFO6105-FinalProject/MetaData/hour.csv',
 'test_path': None,
 'max_models': 9,
 'run_time': 1000,
 'run_id': 'am1d1',
 'scale': False,
 'classification': False,
 'model_path': None,
 'balance': False,
 'balance_threshold': 0.2,
 'project': None,
 'end_time': 1555639702.935195,
 'execution_time': 1218.6394979953766,
 'run_path': '/Users/newzysharma/Desktop/Desktop/info6105/INFO6105-FinalProject/INFO6105-FinalProject/MetaData/am1d1',
 'nthreads': 1,
 'min_mem_size': 1,
 'analysis': 0,
 'X': ['instant',
  'season',
  'yr',
  'mnth',
  'hr',
  'holiday',
  'weekday',
  'workingday',
  'weathersit',
  'casual',
  'registered',
  'dteday',
  'temp',
  'atemp',
  'hum',
  'windspeed'],
 'variables': {'instant': 'int',
  'dteday': 'time',
  'season'

In [75]:
# Save logs
h2o.download_all_logs(dirname=logs_path, filename=logfile)

Writing H2O logs to /Users/newzysharma/Desktop/Desktop/info6105/INFO6105-FinalProject/INFO6105-FinalProject/MetaData/am1d1/logs/am1d1_autoh2o_log.zip


'/Users/newzysharma/Desktop/Desktop/info6105/INFO6105-FinalProject/INFO6105-FinalProject/MetaData/am1d1/logs/am1d1_autoh2o_log.zip'

In [73]:
def predictions_test(mod,test,run_id):
    mod_perf=mod_best.model_performance(test)          
    stats_test={}
    stats_test=model_performance_stats(mod_perf)
    print(stats_test)   
    n=run_id+'_test_stats.json'
    dict_to_json(stats_test,n) 
    try:
      cf=mod_perf.confusion_matrix()
      print(cf)   
#      cf=mod_perf.confusion_matrix(metrics=["f1","f2","f0point5","accuracy","precision","recall","specificity","absolute_mcc","min_per_class_accuracy","mean_per_class_accuracy"])
      cf_df=cf.table.as_data_frame()
      print(cf_df)    
      cf_df.to_csv(run_id+'_test_confusion_matrix.csv')
    except:
      pass
    predictions = mod_best.predict(test)    
    predictions_df=test.cbind(predictions).as_data_frame() 
    predictions_df.to_csv(run_id+'_predictions.csv')
    return predictions

In [74]:
h2o.cluster().shutdown()

H2O session _sid_b4e1 closed.
